In [1]:
import pandas as pd
from sklearn.datasets import make_regression
import numpy as np
import os
import seaborn as sns
import pandas as pd
from math import e,sqrt
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.optimize import leastsq
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import datetime
import statistics
import math
import os
import sys
import statistics
#sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")

from sklearn.svm import SVR
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from scipy.stats import randint as sp_randint


In [3]:
from pyFTS.common import Util
from pyFTS.benchmarks import Measures

In [4]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [5]:
# convert series to supervised learning
def series_to_supervised_miso(data, n_in, n_out, endog_var, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df[endog_var].shift(-i))
		if i == 0:
			names += [(endog_var+'(t)')]
		else:
			names += [(endog_var+'(t+%d)' % (i))]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [6]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [7]:
df = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/energydata_complete.csv', sep=',')
data = df.drop(labels=['date','rv1','rv2'], axis=1)
data = clean_dataset(data)
data = series_to_supervised_miso(data, 1, 1,'Appliances')
data.head()

,Appliances(t-1),lights(t-1),T1(t-1),RH_1(t-1),T2(t-1),RH_2(t-1),T3(t-1),RH_3(t-1),T4(t-1),RH_4(t-1),...,RH_8(t-1),T9(t-1),RH_9(t-1),T_out(t-1),Press_mm_hg(t-1),RH_out(t-1),Windspeed(t-1),Visibility(t-1),Tdewpoint(t-1),Appliances(t)
1,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,60.0
2,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,50.0
3,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,50.0
4,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,60.0
5,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,50.0


In [8]:
values = data.values
values.shape

(19734, 27)

In [9]:
values = data.values
train_size=int(len(values)*0.75)
test_size = int(len(values)) - train_size
print(test_size)

4934


In [10]:
data.loc[:,'lights(t-1)':'Tdewpoint(t-1)'].values[:train_size].shape

(14800, 25)

In [14]:
train_X, train_y = data.loc[:,'lights(t-1)':'Tdewpoint(t-1)'].values[:train_size], data['Appliances(t-1)'].values[:train_size]
test_X, test_y = data.loc[:,'lights(t-1)':'Tdewpoint(t-1)'].values[train_size:], data['Appliances(t)'].values[train_size:]

# train_y.reshape(-1)
# test_y.reshape(-1)

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(14800, 25) (14800,) (4934, 25) (4934,)


In [15]:
# Get the number of inputs and outputs from the dataset
df_trainx = pd.DataFrame(train_X)
df_testx = pd.DataFrame(test_X)
df_trainy = pd.DataFrame(train_y)
df_original = pd.DataFrame(test_y)

In [16]:
df_original

,0
0,280.0
1,320.0
2,360.0
3,210.0
4,90.0
...,...
4929,100.0
4930,90.0
4931,270.0
4932,420.0


## RandomizedSearchCV

In [19]:
from scipy import stats
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__C=stats.uniform(0,50),
                     estimator__kernel=['poly', 'rbf','sigmoid'],
                     estimator__epsilon=stats.uniform(0,1))
clf=SVR()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1,n_iter=50)

hyperparameters_tuning = gsc.fit(df_trainx, df_trainy)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Parameters = {'estimator__C': 46.815961459701114, 'estimator__epsilon': 0.7937574989865204, 'estimator__kernel': 'poly'}


In [20]:
hyperparameters_tuning.best_estimator_

MultiOutputRegressor(estimator=SVR(C=46.815961459701114,
                                   epsilon=0.7937574989865204, kernel='poly'))

In [21]:
y_p = tuned_model.predict(df_testx)
columns = df_original.columns
y_p = pd.DataFrame(y_p,columns=columns)
y_p

,0
0,74.703635
1,75.111164
2,75.770495
3,76.299921
4,76.652019
...,...
4929,81.446989
4930,81.322519
4931,81.116215
4932,93.888184


In [23]:
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[]
    }
    
final_result = {
        
         "rmse": [],
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
}

original = df_original.values
forecast = y_p.values

mae = round(mean_absolute_error(original,forecast),3)
r2 = round(r2_score(original,forecast),3)
rmse = round(mean_squared_error(original,forecast,squared=False),3)             
mape = round(mean_absolute_percentage_error(original,forecast),3)

result["rmse"].append(rmse)
result["mae"].append(mae)
result["r2"].append(r2)
result["mape"].append(mape)
        
measures = pd.DataFrame(result)
measures

,rmse,mae,r2,mape
0,88.467,39.129,-0.004,0.318


## MinMaxScaler

In [24]:
from sklearn.preprocessing import MinMaxScaler


In [25]:
df = pd.read_csv('/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/energydata_complete.csv', sep=',')
data = df.drop(labels=['date','rv1','rv2'], axis=1)
data = clean_dataset(data)
data = series_to_supervised_miso(data, 1, 1,endog_var='Appliances')
values = data.values

In [26]:
data.head()

,Appliances(t-1),lights(t-1),T1(t-1),RH_1(t-1),T2(t-1),RH_2(t-1),T3(t-1),RH_3(t-1),T4(t-1),RH_4(t-1),...,RH_8(t-1),T9(t-1),RH_9(t-1),T_out(t-1),Press_mm_hg(t-1),RH_out(t-1),Windspeed(t-1),Visibility(t-1),Tdewpoint(t-1),Appliances(t)
1,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,60.0
2,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,50.0
3,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,50.0
4,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,60.0
5,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,50.0


In [27]:
cols = data.columns
list_cols = list(cols)

In [28]:
train_X, train_y = data.loc[:,'lights(t-1)':'Tdewpoint(t-1)'].values[:train_size], data['Appliances(t-1)'].values[:train_size]
test_X, test_y = data.loc[:,'lights(t-1)':'Tdewpoint(t-1)'].values[train_size:], data['Appliances(t)'].values[train_size:]

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(14800, 25) (14800,) (4934, 25) (4934,)


In [29]:
test_y

array([280., 320., 360., ..., 270., 420., 430.])

In [30]:
from sklearn.preprocessing import MinMaxScaler
scalerx = MinMaxScaler()
scalery = MinMaxScaler()

In [31]:
# # Get the number of inputs and outputs from the dataset
df_trainx = pd.DataFrame(train_X)
df_testx = pd.DataFrame(test_X)
df_trainy = pd.DataFrame(train_y)
df_original = pd.DataFrame(test_y)

In [32]:
df_original

,0
0,280.0
1,320.0
2,360.0
3,210.0
4,90.0
...,...
4929,100.0
4930,90.0
4931,270.0
4932,420.0


In [33]:
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx_scaled = scalerx.transform(df_testx)
df_trainy_scaled = scalery.fit_transform(df_trainy)


In [34]:
from scipy import stats
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
# Define the hyperparameter configuration space
hyperparameters = dict(
                     estimator__C=stats.uniform(0,50),
                     estimator__kernel=['poly', 'rbf','sigmoid'],
                     estimator__epsilon=stats.uniform(0,1))
clf=SVR()
model=MultiOutputRegressor(clf)
gsc = RandomizedSearchCV(model,hyperparameters,cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1,n_iter=30)

hyperparameters_tuning_nor = gsc.fit(df_trainx_scaled, df_trainy_scaled)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning_nor.best_estimator_

Best Parameters = {'estimator__C': 46.815961459701114, 'estimator__epsilon': 0.7937574989865204, 'estimator__kernel': 'poly'}


In [40]:
hyperparameters_tuning_nor.best_estimator_

MultiOutputRegressor(estimator=SVR(C=14.259799162081748,
                                   epsilon=0.20041861401553474))

In [41]:
y_p = tuned_model.predict(df_testx_scaled)
columns = df_original.columns
y_p = pd.DataFrame(y_p,columns=columns)
y_p

,0
0,0.290686
1,0.285771
2,0.290158
3,0.283311
4,0.285251
...,...
4929,0.090047
4930,0.097483
4931,0.106421
4932,0.100323


In [42]:
y_p_inv = scalery.inverse_transform(y_p)
columns = df_original.columns
y_p_inv = pd.DataFrame(y_p_inv,columns=columns)
y_p_inv

,0
0,321.034495
1,315.775452
2,320.469144
3,313.143261
4,315.218498
...,...
4929,106.350542
4930,114.306834
4931,123.870443
4932,117.345320


In [43]:
df_original

,0
0,280.0
1,320.0
2,360.0
3,210.0
4,90.0
...,...
4929,100.0
4930,90.0
4931,270.0
4932,420.0


In [44]:
result = {
        
         "rmse": [],  
         "mae": [],
         "r2": [],
         "mape":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[]
    }
 
original = df_original.values
forecast = y_p_inv.values
mae = round(mean_absolute_error(original,forecast),3)
r2 = round(r2_score(original,forecast),3)
rmse = round(mean_squared_error(original,forecast,squared=False),3)             
mape = round(mean_absolute_percentage_error(original,forecast),3)

result["rmse"].append(rmse)
result["mae"].append(mae)
result["r2"].append(r2)
result["mape"].append(mape)
        
measures = pd.DataFrame(result)
measures 

,rmse,mae,r2,mape
0,192.273,159.666,-3.743,2.374


# Model Design SVR

In [72]:
def create_and_fit_model(X_train,y_train):
    #MultiOutputRegressor(estimator=SVR(C=31.59275568053482,epsilon=0.4161116186137991, kernel='poly'))
    MultiOutputRegressor(estimator=SVR(C=31.319000850474215,epsilon=0.16642613101735404, kernel='poly'))
    #MultiOutputRegressor(estimator=SVR(C=14.259799162081748,epsilon=0.20041861401553474))
    #MultiOutputRegressor(estimator=SVR(C=46.815961459701114,epsilon=0.7937574989865204, kernel='poly'))
    clf.fit(X_train, y_train)
    return clf

In [73]:
def forecast_model(clf, X_test):
    forecast = clf.predict(X_test)
    return forecast

In [84]:
def sliding_window(data,n_windows,train_size):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape":[],
         "nrmse": []
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape":[],
         "nrmse": []
    }

    tam = len(data)
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            scaler = StandardScaler()
            scalery = StandardScaler()

            
            Xtrain = ttrain.loc[:,'lights(t-1)':'Tdewpoint(t-1)']
            ytrain = ttrain.loc[:,'Appliances(t-1)']
            Xtest = ttest.loc[:,'lights(t-1)':'Tdewpoint(t-1)']
            ytest = ttest.loc[:,'Appliances(t)']

#             df_trainx = pd.DataFrame(Xtrain)
#             df_testx = pd.DataFrame(Xtest)
#             df_trainy = pd.DataFrame(ytrain)
#             df_original = pd.DataFrame(ytest)
            
#             Xtrain = scaler.fit_transform(df_trainx)
#             ytrain = scalery.fit_transform(df_trainy)
#             Xtest = scaler.transform(df_testx)
            #ytest = scalery.transform(ttest['Appliances(t)'])
            
           
            Xtrain = scaler.fit_transform(ttrain.loc[:,'lights(t-1)':'Tdewpoint(t-1)'])
            ytrain = scalery.fit_transform(ttrain.loc[:,'Appliances(t-1)'].values)
            Xtest = scaler.transform(ttest.loc[:,'lights(t-1)':'Tdewpoint(t-1)'])
            
            
            model = create_and_fit_model(Xtrain,ytrain)
            forecast = forecast_model(model, Xtest)
             
            df_forecats_columns = ['Appliances(t)']
            
            columns = list(df_forecats_columns)
            
            prediction = scaler.inverse_transform(forecast) 
            #prediction = forecast
            df_forecast = pd.DataFrame(prediction,columns=columns)
            
            #ytest_metric = scaler.inverse_transform(y_test.reshape(y_test.shape[0], y_test.shape[2]))
            ytest_metric = ttest.loc[:,'Appliances(t)'].values
            df_original = pd.DataFrame(df_original,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #mae = round(mean_absolute_error(original,forecast),3)
                #r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["smape"].append(mae)
                result["r2"].append(r2)
                result["window"].append(ct)
                
                fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
                ax.plot(original, label='Original')
                ax.plot(forecast, label='Forecast')
                handles, labels = ax.get_legend_handles_labels()
                lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
                plt.show()
        
    measures = pd.DataFrame(result)
    return measures

In [85]:
n_windows = 30
train_size = 0.75 

model_svr = sliding_window(data,n_windows,train_size)

--------------------
training window 0


/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


ValueError: operands could not be broadcast together with shapes (164,) (25,) (164,) 

In [76]:
model_svr 

,window,rmse,mape,mae,r2,smape,nrmse
0,0,134.280,34.509,159.666,-3.743,159.666,0.280
1,657,155.731,42.429,159.666,-3.743,159.666,0.192
2,1314,191.341,43.565,159.666,-3.743,159.666,0.231
3,1971,21.097,34.515,159.666,-3.743,159.666,0.211
4,2628,95.462,27.906,159.666,-3.743,159.666,0.167
5,3285,69.455,44.417,159.666,-3.743,159.666,0.169
6,3942,142.147,47.619,159.666,-3.743,159.666,0.195
7,4599,94.689,45.955,159.666,-3.743,159.666,0.141
8,5256,65.382,40.210,159.666,-3.743,159.666,0.133
9,5913,138.436,47.265,159.666,-3.743,159.666,0.182


In [77]:
measures.to_csv (r'win_svr_aec.csv', index = False, header=True)

In [78]:
measures = model_svr

In [79]:
final_result = {
     "rmse": [],
     "mae": [],
     "mape": [],
     "smape": [],
     "nrmse":[],
     "smape":[],
     "rmse_std": [],
     "mae_std": [],
     "mape_std": [],
     "smape_std": [],
     "nrmse_std": [],
     "smape_std":[]
}

final_result["rmse"].append(round(statistics.mean(measures['rmse']),3))
final_result["mape"].append(round(statistics.mean(measures['mape']),3))
final_result["smape"].append(round(statistics.mean(measures['smape']),3))
final_result["mae"].append(round(statistics.mean(measures['mae']),3))
final_result["nrmse"].append(round(statistics.mean(measures['nrmse']),3))

final_result["rmse_std"].append(round(statistics.stdev(measures['rmse']),3))
final_result["mape_std"].append(round(statistics.stdev(measures['mape']),3))
final_result["smape_std"].append(round(statistics.stdev(measures['smape']),3))
final_result["mae_std"].append(round(statistics.stdev(measures['mae']),3))
final_result["nrmse_std"].append(round(statistics.stdev(measures['nrmse']),3))
        
final_measures = pd.DataFrame(final_result)

print("Mean Statistics SRV (test): ")
final_measures

Mean Statistics SRV (test): 


,rmse,mae,mape,smape,nrmse,rmse_std,mae_std,mape_std,smape_std,nrmse_std
0,97.063,159.666,37.837,159.666,0.188,48.829,0.0,7.364,0.0,0.057


In [87]:
final_measures.to_csv (r'svr_aec_with_std.csv', index = False, header=True)

In [ ]:
# df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
#                                    'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
#                                    'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
#                                    'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
#                                    'Visibility(t)', 'Tdewpoint(t)']

# columns = list(df_forecats_columns)

# final_result = {
#     "variable": [],
#     "rmse": [],
#     "mae": [],
#     "mape": [],
#     "r2": [],
#     "smape": [],
#     "nrmse": [],
#     "rmse_std": [],
#     "mae_std": [],
#     "mape_std": [],
#     "r2_std": [],
#     "smape_std": [],
#     "nrmse_std": []
# }

# model_svr.to_csv (r'all_windows_svr_uci_appliances.csv', index = False, header=True)

# measures = model_svr
# var = measures.groupby("variable")

# for col in columns:
    
#     var_agr = var.get_group(col)
           
#     rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
#     mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
#     mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
#     r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
#     smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
#     nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
#     rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
#     mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
#     mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
#     r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
#     smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
#     nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

#     final_result["variable"].append(col)
#     final_result["rmse"].append(rmse)
#     final_result["mape"].append(mape)
#     final_result["mae"].append(mae)
#     final_result["r2"].append(r2)
#     final_result["smape"].append(smape)
#     final_result["nrmse"].append(nrmse)
    
#     final_result["rmse_std"].append(rmse_std)
#     final_result["mape_std"].append(mape_std)
#     final_result["mae_std"].append(mae_std)
#     final_result["r2_std"].append(r2_std)
#     final_result["smape_std"].append(smape_std)
#     final_result["nrmse_std"].append(nrmse_std)
        
#     #print(f'Results: {(col,rmse,mae,r2)}')
        
        
# final_measures = round(pd.DataFrame(final_result),3)

In [ ]:
#final_measures